In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# import required packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils, to_categorical

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# remove id column from train df
train_df.pop('id')

# get target labels for train df
train_labels = train_df.pop('species')

# remove id column from test df
test_data_id = test_df.pop('id')

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_arr = train_df.values
test_arr = test_df.values

print(train_arr.shape)
print(test_arr.shape)

In [ ]:
print(train_labels.shape)

In [ ]:
# encode training labels into binary matrix
labelEncoder = LabelEncoder()
train_labels_list = list(train_labels)
transformed_train_labels = labelEncoder.fit(train_labels_list).transform(train_labels_list)
train_labels_arr = to_categorical(transformed_train_labels)

In [ ]:
train_labels_arr.shape

In [ ]:
# split dataset into train and validation set
X_train, X_val, Y_train, Y_val = train_test_split(train_arr, train_labels_arr, test_size= 0.2)

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

In [ ]:
# define a model
model = Sequential()

model.add(Dense(128, kernel_initializer="uniform", input_dim= 192, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(99, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# train neural network on train dataset and validate on validation set
model_history = model.fit(x=X_train,y=Y_train, epochs=500, batch_size= 32, validation_data=(X_val, Y_val), verbose=1)

In [ ]:
# compute predictions on test dataset
predictions = model.predict(test_arr, batch_size=32, verbose=1)
computed_predictions = np.argmax(predictions, axis=1)

In [ ]:
# plot graph of variation of cost with number of iterations

plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('model loss')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# plot variation of accuracy with epochs
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('model accuracy')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
#print(train_labels.unique()[:4])
submission = pd.DataFrame(predictions, columns = train_labels.unique())
submission['id']=test_data_id
submission.reset_index(drop=True, inplace=True)
submission.head()

In [ ]:
submission.to_csv('submission.csv',index=False)